# Percent College Graduates Prediction
---
* Input: `'../data/census_tract_feats.csv'`
* Output: Regression model that predicts the percentage of the age 25+ population that has a college degree in a given census tract

In [1]:
import pandas as pd

# Read in original data
df = pd.read_csv('../data/census_tract_feats.csv')
df.head()

,geoID,Total Population:,Population Density (Per Sq. Mile),Total Population: Male,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,...,"Pct. Households: $100,000 to $124,999","Pct. Households: $125,000 to $149,999","Pct. Households: $150,000 to $199,999","Pct. Households: $200,000 or More",Pct. Families below poverty level,Pct. Population for Whom Poverty Status Is Determined: Under 1.00 (Doing Poorly),Pct. Population for Whom Poverty Status Is Determined: 1.00 to 1.99 (Struggling),Pct. Population for Whom Poverty Status Is Determined: Under 2.00 (Poor or Struggling),Number of Accessible Universities,Education Desert
0,1001020100,1845,487.1106,899,1636,96,0,22,0,0,...,0.074271,0.079576,0.090186,0.021220,0.120287,0.106775,0.117073,0.223848,5,0
1,1001020200,2172,1684.0130,1167,913,1184,0,22,14,7,...,0.089400,0.040868,0.015326,0.000000,0.182903,0.224138,0.220588,0.444726,5,0
2,1001020300,3385,1638.9340,1533,2078,896,19,25,12,272,...,0.078968,0.039093,0.027365,0.000000,0.100363,0.146529,0.280650,0.427179,5,0
3,1001020400,4267,1731.4730,2001,3443,356,20,10,0,320,...,0.104631,0.032590,0.038308,0.005146,0.014617,0.022967,0.182564,0.205531,5,0
4,1001020500,9965,2264.4190,5054,7817,1638,0,305,0,0,...,0.178827,0.048164,0.054125,0.020029,0.084132,0.122349,0.133108,0.255457,5,0


In [2]:
# Count education deserts (1), non education deserts (0)
df['Education Desert'].value_counts()

0    63515
1    10230
Name: Education Desert, dtype: int64

In [3]:
# select columns to keep
train_feats = [
    'geoID',
    
    # demographics
    'Total Population:',
    'Population Density (Per Sq. Mile)',
    'Pct. Male', 'Pct. White Alone', 'Pct. Black or African American Alone', 'Pct. American Indian and Alaska Native Alone', 'Pct. Asian Alone', 'Pct. Native Hawaiian and Other Pacific Islander Alone', 'Pct. Some Other Race Alone', 'Pct. Two or More Races', 
    'Pct. Under 5 Years', 'Pct. 5 to 9 Years', 'Pct. 10 to 14 Years', 'Pct. 15 to 17 Years', 'Pct. 18 to 24 Years', 'Pct. 25 to 34 Years', 'Pct. 35 to 44 Years', 'Pct. 45 to 54 Years', 'Pct. 55 to 64 Years', 'Pct. 65 to 74 Years', 'Pct. 75 to 84 Years', 'Pct. 85 Years and Over',
    
    # education - omit post-college features (leakage)
    'Pct. Students enrolled in private school',
    'Pct. Population 25 Years and Over: Less than High School',
    'Pct. Population 25 Years and Over: High School Graduate (Includes Equivalency)',
    "Pct. Population 25 Years and Over: Bachelor's Degree",
    
    # employment
    'Pct. Pop 16+ not in labor force',
    'Pct. Pop 16+ in armed forces',
    'Pct. Pop 16+ unemployed',
    
    # household income
    'Median Gross Rent',
    'Median Household Income (In 2017 Inflation Adjusted Dollars)',
    'Pct. Households: Less than $10,000', 'Pct. Households: $10,000 to $14,999', 'Pct. Households: $15,000 to $19,999', 'Pct. Households: $20,000 to $24,999', 'Pct. Households: $25,000 to $29,999', 'Pct. Households: $30,000 to $34,999', 'Pct. Households: $35,000 to $39,999', 'Pct. Households: $40,000 to $44,999', 'Pct. Households: $45,000 to $49,999', 'Pct. Households: $50,000 to $59,999', 'Pct. Households: $60,000 to $74,999', 'Pct. Households: $75,000 to $99,999', 'Pct. Households: $100,000 to $124,999', 'Pct. Households: $125,000 to $149,999', 'Pct. Households: $150,000 to $199,999', 'Pct. Households: $200,000 or More', 
    
    # poverty
    'Pct. Families below poverty level', 'Pct. Population for Whom Poverty Status Is Determined: Under 1.00 (Doing Poorly)', 'Pct. Population for Whom Poverty Status Is Determined: 1.00 to 1.99 (Struggling)', 'Pct. Population for Whom Poverty Status Is Determined: Under 2.00 (Poor or Struggling)',
    
    # education desert
    'Education Desert'
]

# Only keep selected features
df_train = df[train_feats]

# only keep non-education deserts -- education oases? (these are the census blocks we want to regress on)
df_train = df_train[df_train['Education Desert'] == 0]

# store a separate df_predict to predict pct_bachelors on education deserts
df_predict = df[train_feats]
df_predict = df_predict[df_predict['Education Desert'] == 1]
df_predict = df_predict.drop(labels=['Education Desert', "Pct. Population 25 Years and Over: Bachelor's Degree"], axis=1)

# Get training data
X = df_train.drop(labels=['geoID', 'Education Desert', "Pct. Population 25 Years and Over: Bachelor's Degree"], axis=1).values
y = df_train["Pct. Population 25 Years and Over: Bachelor's Degree"]

print('X shape: ', X.shape)
print('y shape: ', y.shape)

df_train.head()

X shape:  (63515, 50)
y shape:  (63515,)


,geoID,Total Population:,Population Density (Per Sq. Mile),Pct. Male,Pct. White Alone,Pct. Black or African American Alone,Pct. American Indian and Alaska Native Alone,Pct. Asian Alone,Pct. Native Hawaiian and Other Pacific Islander Alone,Pct. Some Other Race Alone,...,"Pct. Households: $75,000 to $99,999","Pct. Households: $100,000 to $124,999","Pct. Households: $125,000 to $149,999","Pct. Households: $150,000 to $199,999","Pct. Households: $200,000 or More",Pct. Families below poverty level,Pct. Population for Whom Poverty Status Is Determined: Under 1.00 (Doing Poorly),Pct. Population for Whom Poverty Status Is Determined: 1.00 to 1.99 (Struggling),Pct. Population for Whom Poverty Status Is Determined: Under 2.00 (Poor or Struggling),Education Desert
0,1001020100,1845,487.1106,0.487263,0.886721,0.052033,0.000000,0.011924,0.000000,0.000000,...,0.180371,0.074271,0.079576,0.090186,0.021220,0.120287,0.106775,0.117073,0.223848,0
1,1001020200,2172,1684.0130,0.537293,0.420350,0.545120,0.000000,0.010129,0.006446,0.003223,...,0.139208,0.089400,0.040868,0.015326,0.000000,0.182903,0.224138,0.220588,0.444726,0
2,1001020300,3385,1638.9340,0.452880,0.613885,0.264697,0.005613,0.007386,0.003545,0.080355,...,0.046130,0.078968,0.039093,0.027365,0.000000,0.100363,0.146529,0.280650,0.427179,0
3,1001020400,4267,1731.4730,0.468948,0.806890,0.083431,0.004687,0.002344,0.000000,0.074994,...,0.161807,0.104631,0.032590,0.038308,0.005146,0.014617,0.022967,0.182564,0.205531,0
4,1001020500,9965,2264.4190,0.507175,0.784446,0.164375,0.000000,0.030607,0.000000,0.000000,...,0.153314,0.178827,0.048164,0.054125,0.020029,0.084132,0.122349,0.133108,0.255457,0


In [4]:
# Ignore imblearn UserWarning
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from imblearn import FunctionSampler
import imblearn.pipeline
from collections import defaultdict

# Subsampler for GPR
n_subsample = 1000
def subsample(X, y):
    idx = np.random.choice(np.arange(len(X)), n_subsample, replace=False)
    return X[idx], y[idx]

# TODO: Use RandomizedSearchCV for hyperparameter tuning
models = {
    'LinearRegression': make_pipeline(StandardScaler(), LinearRegression()),
    'PolyRegression': make_pipeline(StandardScaler(), PolynomialFeatures(degree=2), LinearRegression()),
    'SVR': make_pipeline(StandardScaler(), SVR(kernel='rbf')),
    'RandomForest': make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100)),
    'MLP': make_pipeline(StandardScaler(), MLPRegressor()),
    'GaussianProcess': imblearn.pipeline.make_pipeline(StandardScaler(), FunctionSampler(subsample), GaussianProcessRegressor())
}

# Cross validation
metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv_results = {}
for name, model in models.items():
    results = cross_validate(model, X, y, cv=5, scoring=metrics, n_jobs=1)
    cv_results[name] = results

# Convert CV results to Pandas dataframe
pd_results = defaultdict(dict)
for name, results in cv_results.items():
    test_metrics = ['test_' + metric for metric in metrics]
    for test_metric in test_metrics:
        pd_results[name][test_metric] = np.mean(results[test_metric])
    pd_results[name]['fit_time'] = np.mean(results['fit_time'])
        
# Display results
# TODO: plots of predicted vs. true
results_df = pd.DataFrame.from_dict(pd_results, orient='index')
results_df

/Users/lucashu/anaconda/envs/sony-ml/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


,test_r2,test_neg_mean_squared_error,test_neg_mean_absolute_error,fit_time
GaussianProcess,-3.239965,-0.047138,-0.189617,0.241476
LinearRegression,0.836100,-0.001817,-0.031175,0.387712
MLP,0.825312,-0.001951,-0.032185,4.585099
PolyRegression,0.791487,-0.002291,-0.030200,25.161612
RandomForest,0.845351,-0.001717,-0.030000,232.651060
SVR,0.799121,-0.002228,-0.034990,10.418445


In [5]:
from joblib import dump, load

# Re-train best model on all data
model = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100))
model.fit(X, y)

dump(model, '../models/rf-regressor.joblib')

['../models/rf-regressor.joblib']

In [7]:
# Generate pct_bachelors_degree predictions on education deserts
df_predict['pred_pct_bachelors'] = model.predict(df_predict.drop(['geoID', 'pred_pct_bachelors'], axis=1, errors='ignore'))
predictions = df_predict[['geoID', 'pred_pct_bachelors']]
predictions.head()

,geoID,pred_pct_bachelors
12,1003010100,0.083493
13,1003010200,0.093392
15,1003010400,0.131523
25,1003010905,0.117125
26,1003010906,0.089177


In [8]:
# save results in CSV
predictions.to_csv('../data/pct_bachelors_predictions.csv', header=['geoID', 'pred_pct_bachelors'], index=False)